In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (24, 16)
import datetime
import seaborn as sns
import os

In [65]:
def create_resampled_data(path, og_path = "data/D1NAMO/"):
    for subject_type in os.listdir(path):
        for subject_ID in os.listdir(f"{path}/{subject_type}"):
            for day in os.listdir(f"{path}/{subject_type}/{subject_ID}/sensor_data"):
                try:
                    df = pd.read_csv(f"{og_path}/{subject_type}/{subject_ID}/sensor_data/{day}/{day}_Summary.csv")
                    
                    accel = pd.read_csv(f"{og_path}/{subject_type}/{subject_ID}/sensor_data/{day}/{day}_Accel.csv")
                    breathing = pd.read_csv(f"{og_path}/{subject_type}/{subject_ID}/sensor_data/{day}/{day}_Breathing.csv")  
                    ecg = pd.read_csv(f"{og_path}/{subject_type}/{subject_ID}/sensor_data/{day}/{day}_ECG.csv")
                    current_df = join_files(accel,breathing, ecg)
                    current_df.to_csv(f"{path}/{subject_type}/{subject_ID}/sensor_data/{day}/{day}_resampled_data.csv")
                except FileNotFoundError:
                    print(f"{og_path}/{subject_type}/{subject_ID}/sensor_data/{day}/")

In [63]:
def join_files(accel,breathing, ecg):
    
    accel["Time"] = pd.to_datetime(accel["Time"], format="%d/%m/%Y %H:%M:%S.%f")
    breathing["Time"] = pd.to_datetime(breathing["Time"], format="%d/%m/%Y %H:%M:%S.%f")
    ecg["Time"] = pd.to_datetime(ecg["Time"], format="%d/%m/%Y %H:%M:%S.%f")
    
    new_df = accel.set_index("Time").resample("s").mean().copy().rename(
        columns={"Vertical":"V_Mean","Lateral":"L_Mean","Sagittal":"S_Mean"})
    
    accel_std = accel.set_index("Time").resample("s").std().copy().rename(
        columns={"Vertical":"V_STD","Lateral":"L_STD","Sagittal":"S_STD"})
    new_df = new_df.join(accel_std)
    
    breathing_mean = breathing.set_index("Time").resample("s").mean().copy().rename(
        columns={"BreathingWaveform":"B_WF_Mean"})
    new_df = new_df.join(breathing_mean)
    
    breathing_std = breathing.set_index("Time").resample("s").std().copy().rename(
        columns={"BreathingWaveform":"B_WF_STD"})
    new_df = new_df.join(breathing_std)
    
    ecg_mean = ecg.set_index("Time").resample("s").mean().copy().rename(
        columns={"EcgWaveform":"ECG_WF_Mean"})
    new_df = new_df.join(ecg_mean)
    
    ecg_std = ecg.set_index("Time").resample("s").std().copy().rename(
        columns={"EcgWaveform":"ECG_WF_STD"})
    new_df = new_df.join(ecg_std)

    
    return new_df

In [66]:
create_resampled_data(path, og_path = "data/D1NAMO/")

data/D1NAMO//healthy_subset/007/sensor_data/2014_10_01-07_50_37/


In [ ]:
def git 